# Reconhecimento de Captchas

**Aluno:** Pedro de Oliveira Guedes

**Semestre:** 2024.2

## Introdução
Este trabalho tem por objetivo explorar diferentes técnicas para realizar o reconhecimento de caracteres em imagens, mais especificamente em captchas. Para isso, será utilizado um dataset com 10 mil captchas disponibilizado nos arquivos de aula da disciplina de **Introdução à Computação Visual**.

As técnicas empregadas são duas:

1. **HOG + SVM**: ***Histogram of Oriented Gradients***, ou por abreviação ***HOG***, é uma técnica que extrai características de uma imagem baseando-se na orientação dos gradientes. Com as características extraídas, será treinado um modelo de classificação baseado em ***Support Vector Machine***, ou por abreviação ***SVM***, que utilizará as características extraídas e os rótulos das imagens para classificar os caracteres.

2. **CNN**: ***Convolutional Neural Network***, ou por abreviação ***CNN***, é uma técnica de aprendizado profundo que utiliza camadas convolucionais para extrair características de uma imagem. Com as características extraídas, será treinado um modelo de classificação que utilizará as características extraídas e os rótulos das imagens para classificar os caracteres.

Como as imagens necessárias para treino, validação e teste, bem como os rótulos de cada uma delas, já foram fornecidas e já estão separadas, a organização deste trabalho será feita através das seguintes seções:

1. **Funções Comuns**: Nesta seção, serão definidas funções comuns que serão utilizadas em ambas as técnicas, como a avaliação dos modelos gerados.

2. **HOG + SVM**: Nesta seção, será feita a extração das características das imagens utilizando a técnica ***HOG***, bem como o treinamento do modelo de classificação utilizando a técnica ***SVM***. Ao fim, será feita a avaliação do modelo gerado.

3. **CNN**: Nesta seção, será feita a extração das características das imagens utilizando a técnica ***CNN***, bem como o treinamento do modelo de classificação. Ao fim, será feita a avaliação do modelo gerado.

4. **Conclusão**: Nesta seção, será feita uma comparação entre as técnicas utilizadas, bem como uma análise dos resultados obtidos.

## Funções Comuns
Cada uma das funções comuns terá seu próprio subtítulo, com uma breve descrição do objetivo.

### Download e extração do Dataset
A pasta contendo os captchas separados em conjuntos de treino, validação e teste foi disponibilizada no Google Drive. A função abaixo faz o download para a pasta local e extrai o conteúdo do arquivo.

In [1]:
import os
import requests
import zipfile


def save_response_content(response):
    CHUNK_SIZE = 32768

    with open("captcha_10k.zip", "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:
                f.write(chunk)


def extract_captchas_zip():
    with zipfile.ZipFile("captcha_10k.zip", 'r') as zip_ref:
        zip_ref.extractall()

    os.remove("captcha_10k.zip")


def extract_query_string(response_content: str) -> dict:
    content_id = response_content.split('<input type="hidden" name="id" value="')[1].split('">')[0]
    export = response_content.split('<input type="hidden" name="export" value="')[1].split('">')[0]
    confirm = response_content.split('<input type="hidden" name="confirm" value="')[1].split('">')[0]
    uuid = response_content.split('<input type="hidden" name="uuid" value="')[1].split('">')[0]

    return {
        'id': content_id,
        'export': export,
        'confirm': confirm,
        'uuid': uuid
    }


def download_captchas_from_google_drive():
    if os.path.exists("captcha_10k"):
        return

    BASE_URL = "https://drive.google.com/uc?export=download"
    DOWNLOAD_BASE_URL = "https://drive.usercontent.google.com/download"

    session = requests.Session()
    response = session.get(BASE_URL, params={'id': "1mZlwRlSOeNBjFYFzI25RgKvomKRHeJ1Y"}, stream=True)
    
    query_string = extract_query_string(response.text)
    
    response = session.get(DOWNLOAD_BASE_URL, params=query_string, stream=True)
    
    save_response_content(response)

    extract_captchas_zip()

### Carregamento do Dataset
A função abaixo passa por todo o dataset de captchas, armazenando as imagens e os rótulos em um dicionário. Os dicionários serão agrupados em três listas separadas, uma para cada conjunto de dados: treino, validação e teste.

In [ ]:
import cv2 as opencv


def load_captchas_dataset() -> dict:
    download_captchas_from_google_drive()

    TRAIN = "train"
    TEST = "test"
    VALIDATION = "validation"
    FOLDERS = [TRAIN, TEST, VALIDATION]

    all_captchas = {}
    for filename in os.listdir("captcha_10k/labels"):
        with open(f"captcha_10k/labels/{filename}", 'r') as f:
            all_captchas[filename] = {"label": f.read().strip()}
    
    for folder in FOLDERS:
        for filename in os.listdir(f"captcha_10k/{folder}"):
            image = opencv.imread(f"captcha_10k/{folder}/{filename}", opencv.IMREAD_GRAYSCALE)
            all_captchas[filename]["image"] = image / 255.0
            all_captchas[filename]["type"] = folder

    train_captchas = [all_captchas[filename] for filename in all_captchas if all_captchas[filename]["type"] == TRAIN]
    test_captchas = [all_captchas[filename] for filename in all_captchas if all_captchas[filename]["type"] == TEST]
    validation_captchas = [all_captchas[filename] for filename in all_captchas if all_captchas[filename]["type"] == VALIDATION]

    return {
        TRAIN: train_captchas,
        TEST: test_captchas,
        VALIDATION: validation_captchas
    }

## Referências
- [**Intuição por trás do algoritmo HOG**](https://www.youtube.com/watch?v=28xk5i1_7Zc)
- [**Intuição por trás do algoritmo SVM - Alternativa**](https://www.youtube.com/watch?v=RaaGoB8XnxM)